# Modelling the price of diamonds

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.graphics.gofplots import qqplot

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression

from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression

from sklearn import metrics as metrics


seed = 42

Loading and inspecting the dataset.

In [ ]:
data = sns.load_dataset("diamonds")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

Fix errors in the data.

In [ ]:
data = data.query('x > 0 and y > 0 and z > 0')

In [ ]:
data.isna().sum()

Split data into training, validation and test datasets.

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, 
                                         shuffle=True, random_state=seed)

train_data, val_data = train_test_split(train_data, test_size=0.2, 
                                        shuffle=True, random_state=seed)

In [ ]:
print("Dataset sizes - train: %d, validation: %d, test: %d" 
      % (len(train_data), len(val_data), len(test_data)))

Explore the relationships in our data.

In [ ]:
num_cols = train_data.select_dtypes(include='number').columns.tolist()
cat_cols = train_data.select_dtypes(include='category').columns.tolist()
num_cols, cat_cols

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, sharey=True, figsize=(12, 8))

fig.suptitle("Relationship between y and predictors", size=16)

for idx, col in enumerate(num_cols):
  sns.scatterplot(data=train_data, x=col, y='price',
                  ax=axes[idx // 4, idx % 4], alpha=0.5)

Remove outliers to visualize variable relationships more easily.

In [ ]:
train_data_num = train_data[num_cols]

perc_1 = train_data_num.quantile(0.01)
perc_99 = train_data_num.quantile(0.99)

filtered_data = train_data_num[
    (train_data_num > perc_1) & 
    (train_data_num < perc_99)]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, sharey=True, figsize=(12, 8))

fig.suptitle("Relationship between y and predictors", size=16)

for idx, col in enumerate(num_cols):
  sns.scatterplot(data=filtered_data, x=col, y='price',
                  ax=axes[idx // 4, idx % 4], alpha=0.5)

Transform variables to create a linear relationship.

In [ ]:
filtered_data['log_price'] = np.log(filtered_data['price'])

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, sharey=True, figsize=(12, 8))

fig.suptitle("Relationship between log(y) and predictors", size=16)

for idx, col in enumerate(num_cols):
  sns.scatterplot(data=filtered_data, x=col, y='log_price',
                  ax=axes[idx // 4, idx % 4], alpha=0.5)

In [ ]:
filtered_data['log_carat'] = np.log(filtered_data['carat'])
filtered_data['log_x'] = np.log(filtered_data['x'])
filtered_data['log_y'] = np.log(filtered_data['y'])
filtered_data['log_z'] = np.log(filtered_data['z'])

num_cols = ['log_carat', 'log_x', 'log_y', 'log_z']

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, sharey=True, figsize=(12, 4))

fig.suptitle("Relationship between log(y) and predictors", size=16)

for idx, col in enumerate(num_cols):
  sns.scatterplot(data=filtered_data, x=col, y='log_price',
                  ax=axes[idx], alpha=0.5)

Check for multicolinearity problems.

In [ ]:
train_data_copy = train_data.copy()

In [ ]:
train_data_copy['log_price'] = np.log(train_data['price'])
train_data_copy['log_carat'] = np.log(train_data['carat'])
train_data_copy['log_x'] = np.log(train_data['x'])
train_data_copy['log_y'] = np.log(train_data['y'])
train_data_copy['log_z'] = np.log(train_data['z'])

train_data_copy.drop(columns=['carat', 'x', 'y', 'z', 'price'], inplace=True)

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(train_data_copy.corr(), annot=True, linewidths=2, cmap='winter')

plt.title("Correlations between numerical variables", size=16)
plt.show()

Analyzing and selecting categorical variables.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(15, 4))

for i, col in enumerate(cat_cols):
  sns.histplot(train_data_copy[col], ax=axes[i])
  axes[i].set_title("%s counts by category" % (col))

In [ ]:
train_data_copy.head()

In [ ]:
for col in cat_cols:
  codes, uniques = pd.factorize(train_data_copy[col])
  train_data_copy[col] = codes

In [ ]:
train_data_copy.head()

In [ ]:
discrete = [True] * 3 + [False] * 7

In [ ]:
info = mutual_info_regression(train_data_copy, train_data_copy['log_price'], 
                              discrete_features=discrete)

info = pd.Series(info, name='Mutual regression scores', 
                 index=train_data_copy.columns.tolist())

info.sort_values(ascending=False, inplace=True)

In [ ]:
info

Building our models.

In [ ]:
X_train = train_data.drop(columns=['price'])
y_train = train_data['price']

X_val = val_data.drop(columns=['price'])
y_val = val_data['price']

X_test = test_data.drop(columns=['price'])
y_test = test_data['price']

In [ ]:
def evaluate_model(model, X, y):
  y_pred = model.predict(X)
  r2 = metrics.r2_score(y, y_pred)
  mse = metrics.mean_squared_error(y, y_pred)
  rmse = np.sqrt(mse)
  return r2, mse, rmse

Model #1: $\ln(\text{price}) = \beta_0 + \beta_1 \ln(\text{carat}) + \beta_2 \text{clarity}$

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", FunctionTransformer(np.log), ['carat']),
        ("cat", OneHotEncoder(drop='first'), ['clarity']),
    ],
    remainder='drop'
)
ttr = TransformedTargetRegressor(
    regressor=LinearRegression(), func=np.log, inverse_func=np.exp)

model_1 = make_pipeline(preprocessor, ttr)

In [ ]:
model_1.fit(X_train, y_train)

In [ ]:
r2, mse, rmse = evaluate_model(model_1, X_train, y_train)
print("Training R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

In [ ]:
r2, mse, rmse = evaluate_model(model_1, X_val, y_val)
print("Validation R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

Model #2: $\ln(\text{price}) = \beta_0 + \beta_1 \ln(\text{carat}) + \beta_2 \text{clarity} + \beta_3 \text{color}$

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", FunctionTransformer(np.log), ['carat']),
        ("cat", OneHotEncoder(drop='first'), ['clarity', 'color']),
    ],
    remainder='drop'
)
ttr = TransformedTargetRegressor(
    regressor=LinearRegression(), func=np.log, inverse_func=np.exp)

model_2 = make_pipeline(preprocessor, ttr)

In [ ]:
model_2.fit(X_train, y_train)

In [ ]:
r2, mse, rmse = evaluate_model(model_2, X_train, y_train)
print("Training R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

In [ ]:
r2, mse, rmse = evaluate_model(model_2, X_val, y_val)
print("Validation R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

Model #3: $\ln(\text{price}) = \beta_0 + \beta_1 \ln(\text{carat}) + \beta_2 \text{clarity} + \beta_3 \text{color} + \beta_4 \text{cut}$

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", FunctionTransformer(np.log), ['carat']),
        ("cat", OneHotEncoder(drop='first'), ['clarity', 'color', 'cut']),
    ],
    remainder='drop'
)
ttr = TransformedTargetRegressor(
    regressor=LinearRegression(), func=np.log, inverse_func=np.exp)

model_3 = make_pipeline(preprocessor, ttr)

In [ ]:
model_3.fit(X_train, y_train)

In [ ]:
r2, mse, rmse = evaluate_model(model_3, X_train, y_train)
print("Training R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

In [ ]:
r2, mse, rmse = evaluate_model(model_3, X_val, y_val)
print("Validation R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

Checking our model's assumptions.

In [ ]:
y_pred = model_2.predict(X_val)
log_y_pred = np.log(y_pred)
log_y_val = np.log(y_val)

residuals = log_y_val - log_y_pred

In [ ]:
sns.scatterplot(x=log_y_val, y=log_y_pred, alpha=0.5)

In [ ]:
sns.jointplot(x=log_y_val, y=residuals)

In [ ]:
sns.histplot(residuals, bins=30)

In [ ]:
qqp = qqplot(residuals, line='s')

Evaluating our model on the test dataset.

In [ ]:
r2, mse, rmse = evaluate_model(model_2, X_test, y_test)
print("Test R^2: %.3f, MSE: %.3f, RMSE: %.3f" % (r2, mse, rmse))

In [ ]:
betas = model_2[-1].regressor_.coef_

In [ ]:
fnames = model_2[0].transformers_[1][1].get_feature_names_out()
fnames = ['log_carat'] + fnames.tolist()

In [ ]:
betas = pd.Series(betas, name="Beta parameters", index=fnames)

In [ ]:
betas